<a href="https://colab.research.google.com/github/esheshka/chess/blob/main/chess_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title **Globals**

table = [['.' for i in range(8)] for i in range(8)]
white_figures = ['P', 'R', 'N', 'B', 'Q', 'K']
black_figures = ['p', 'r', 'n', 'b', 'q', 'k']
figures = [*white_figures, *black_figures]
white_move = True
move_count = 1

memory = []
now = 0

In [2]:
def deepcopy(board):
    new_board = []
    for row in board:
        new_row = []
        for element in row:
            new_row.append(element)
        new_board.append(new_row)
    return new_board

In [3]:
def create_table():
    global table
    global white_move
    global move_count
    global memory
    global now

    table = [['.' for i in range(8)] for i in range(8)]

    table[0] = ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
    table[7] = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
    for x in range(8):
        table[1][x] = 'p'
        table[6][x] = 'P'

    table = flip(transpose(table))

    white_move = True
    move_count = 1
    memory = []
    memory.append(deepcopy((table)))
    now = 0

In [4]:
def transpose(matr):
    res = []
    for j in range(8):
        tmp = []
        for i in range(8):
            tmp = tmp + [matr[i][j]]
        res = res + [tmp]

    return res

In [5]:
def flip(matr):
    res = []
    for j in range(8):
        tmp = []
        for i in range(8):
            tmp = tmp + [matr[j][7 - i]]
        res = res + [tmp]

    return res

In [6]:
def draw():
    print(f"\n   A B C D E F G H\n")
    for x in range(8):
        print(8 - x, end='  ')

        for y in range(8):
            print(table[y][7 - x], end=' ')

        print('', 8 - x)

    print(f"\n   A B C D E F G H\n")

In [7]:
def coordinates(text):
    result = [ord(text[0]) - ord('a'), int(text[1]) - 1]

    return result

In [8]:
def move(command):
    if len(command) != 5 or command[0] not in "abcdefghABSDEFGH" or command[1] not in "12345678" or command[2] != "-" or command[3] not in "abcdefghABSDEFGH" or command[4] not in "12345678":
        return "Wrong input format."

    move_from, move_to = map(coordinates, command.split("-"))

    if table[move_from[0]][move_from[1]] == ".":
        return "The piece cannot make the specified move."
    elif white_move and table[move_from[0]][move_from[1]] in black_figures:
        return "The piece cannot make the specified move."
    elif (not white_move) and table[move_from[0]][move_from[1]] in white_figures:
        return "The piece cannot make the specified move."

    possible = possible_moves(move_from[0], move_from[1])
    # print(possible)
    if move_to not in possible:
        return "The piece cannot make the specified move."
    else:
        table[move_to[0]][move_to[1]] = table[move_from[0]][move_from[1]]
        table[move_from[0]][move_from[1]] = '.'

In [9]:
def possible_moves(x, y):
    pos = []
    if table[x][y] in 'pP':
        pos = [*pos, *possible_moves_pawn(x, y)]
    if table[x][y] in 'rRqQ':
        pos = [*pos, *possible_moves_rook(x, y)]
    if table[x][y] in 'bBqQ':
        pos = [*pos, *possible_moves_bishop(x, y)]
    if table[x][y] in 'nN':
        pos = [*pos, *possible_moves_knight(x, y)]
    if table[x][y] in 'kK':
        pos = [*pos, *possible_moves_king(x, y)]


    return pos

In [10]:
#@title **Возможные ходы короля**

def possible_moves_king(x, y):
    pos = []
    for x1 in [-1, 0, 1]:
        for y1 in [-1, 0, 1]:
            if (x1 == 0 and y1 == 0) or (x + x1 not in range(8)) or (y + y1 not in range(8)):
                continue

            if table[x+x1][y+y1] == '.':
                pos.append([x+x1, y+y1])
            elif (table[x+x1][y+y1] in white_figures) == white_move:
                continue
            else:
                pos.append([x+x1, y+y1])

    return pos

In [11]:
#@title **Возможные ходы коня**

def possible_moves_knight(x, y):
    pos = []
    for x1 in [-2, -1, 1, 2]:
        for y1 in [-2, -1, 1, 2]:
            if (x1*x1 + y1*y1 != 5) or (x + x1 not in range(8)) or (y + y1 not in range(8)):
                continue

            if table[x+x1][y+y1] == '.':
                pos.append([x+x1, y+y1])
            elif (table[x+x1][y+y1] in white_figures) == white_move:
                continue
            else:
                pos.append([x+x1, y+y1])

    return pos

In [12]:
#@title **Возможные ходы слона**

def bishop_vectors(x, y, kx, ky):
    pos = []

    for a in range(1, 8):
        if y + ky*a not in range(8) or x + kx*a not in range(8):
            break
        elif table[x+kx*a][y+ky*a] == '.':
            pos.append([x+kx*a, y+ky*a])
        elif (table[x+kx*a][y+ky*a] in white_figures) == white_move:
            break
        else:
            pos.append([x+kx*a, y+ky*a])
            break

    return pos


def possible_moves_bishop(x, y):

    pos = []

    for x1 in [-1, 1]:
        for y1 in [-1, 1]:
            pos = [*pos, *rook_vectors(x, y, x1, y1)]

    return pos

In [13]:
#@title **Возможные ходы ладьи**

def rook_vectors(x, y, kx, ky):
    pos = []

    for a in range(1, 8):
        if y + ky*a not in range(8) or x + kx*a not in range(8):
            break
        elif table[x+kx*a][y+ky*a] == '.':
            pos.append([x+kx*a, y+ky*a])
        elif (table[x+kx*a][y+ky*a] in white_figures) == white_move:
            break
        else:
            pos.append([x+kx*a, y+ky*a])
            break

    return pos


def possible_moves_rook(x, y):
    pos = []

    for x1 in [-1, 1]:
        pos = [*pos, *rook_vectors(x, y, x1, 0)]
    for y1 in [-1, 1]:
        pos = [*pos, *rook_vectors(x, y, 0, y1)]

    return pos

In [14]:
#@title **Возможные ходы пешки**

def possible_moves_pawn(x, y):
    pos = []
    if white_move:
        if y+1 in range(8):
            if x+1 in range(8) and table[x+1][y+1] in black_figures:
                pos.append([x+1, y+1])
            if x-1 in range(8) and table[x-1][y+1] in black_figures:
                pos.append([x-1, y+1])

            if table[x][y+1] not in figures:
                pos.append([x, y+1])
                if y == 1 and table[x][y+2] not in figures:
                    pos.append([x, y+2])
    else:
        if y-1 in range(8):
            if x+1 in range(8) and table[x+1][y-1] in white_figures:
                pos.append([x+1, y-1])
            if x-1 in range(8) and table[x-1][y-1] in white_figures:
                pos.append([x-1, y-1])

            if table[x][y-1] not in figures:
                pos.append([x, y-1])
                if y == 6 and table[x][y-2] not in figures:
                    pos.append([x, y-2])

    return pos

In [19]:
def my_main(command):
    global table
    global white_move
    global move_count
    global memory
    global now

    if command == 'exit':
        return True
    elif command == 'draw':
        draw()
    # elif command == 'next_move':
    #     if now + 1 < len(memory):
    #         now += 1
    #         table = copy.deepcopy(memory[now])
    #         white_move = not white_move
    #         move_count = (now // 2) + 1
    #     else:
    #         print('Error. Type: History out of bounds')
    elif command.split('next_move')[0] == '' and (command.split('next_move')[1] == '' or command.split('next_move ')[1].isdigit()):
        step = 1
        if command.split('next_move')[1] != '':
            step = int(command.split('next_move ')[1])

        if now + step < len(memory):
            now += step
            table = deepcopy(memory[now])
            if step % 2 == 1:
                white_move = not white_move
            move_count = (now // 2) + 1
        else:
            print('Error. Type: History out of bounds.')
    # elif command == 'previous_move':
    #     if now - 1 >= 0:
    #         now -= 1
    #         table = copy.deepcopy(memory[now])
    #         white_move = not white_move
    #         move_count = (now // 2) + 1
    #     else:
    #         print('Error. Type: History out of bounds')
    elif command.split('previous_move')[0] == '' and (command.split('previous_move')[1] == '' or command.split('previous_move ')[1].isdigit()):
        step = 1
        if command.split('previous_move')[1] != '':
            step = int(command.split('previous_move ')[1])

        if now - step >= 0:
            now -= step
            table = deepcopy(memory[now])
            if step % 2 == 1:
                white_move = not white_move
            move_count = (now // 2) + 1
        else:
            print('Error. Type: History out of bounds.')
    else:
        error = move(command)
        if not error:
            if now + 1 < len(memory):
                memory = memory[:now+1]
            memory.append(deepcopy(table))
            now += 1
            white_move = not white_move
            if white_move:
                move_count += 1
        elif error != "ok":
            print('Error. Type:', error)

In [20]:
create_table()

while True:
    print('white'*white_move + 'black'*(not white_move), str(move_count) + ':')
    if my_main(input()):
        break

white 1:
e2-e4
black 1:
e7-e5
white 2:
draw

   A B C D E F G H

8  r n b q k b n r  8
7  p p p p . p p p  7
6  . . . . . . . .  6
5  . . . . p . . .  5
4  . . . . P . . .  4
3  . . . . . . . .  3
2  P P P P . P P P  2
1  R N B Q K B N R  1

   A B C D E F G H

white 2:
previous_move
black 1:
a7-a5
white 2:
draw

   A B C D E F G H

8  r n b q k b n r  8
7  . p p p p p p p  7
6  . . . . . . . .  6
5  p . . . . . . .  5
4  . . . . P . . .  4
3  . . . . . . . .  3
2  P P P P . P P P  2
1  R N B Q K B N R  1

   A B C D E F G H

white 2:
exit
